# BioMed: Information Retrieval - BioMedical Information Retrieval System

---

**Group:**
- Reyes Castro, Didier Yamil (didier.reyes.castro@alumnos.upm.es)
- Rodriguez Fernández, Cristina ()

**Course:** BioMedical Informatics - 2025/26

**Institution:** Polytechnic University of Madrid (UPM)

**Date:** November 2026

---

## Goal

To develop an Information Retrieval system — specifically, a **binary text classifier** — to identify scientific articles in the PubMed database that are related to a given set of abstracts within a defined research topic. In this case, the focus is on a collection of 1,308 manuscripts containing information on the polyphenol composition of various foods.

## Setup and Installation

In [ ]:
# !pip install scikit-learn pandas requests transformers pytorch datasets numpy

In [1]:
import requests
import time
import re

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

/home/didierrc/miniconda3/envs/biomed_ir/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Task 1:** 

Retrieve from PubMed the abstracts associated with each publication in publications.xlsx

(21 minutes with API KEY)

In [ ]:
dataset = pd.read_csv('publications.csv')
dataset

In [ ]:
BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
ESEARCH_URL = BASE_URL + "esearch.fcgi"
FETCH_URL = BASE_URL + "efetch.fcgi"
DS_WITH_PMID = 'publications_pmid.csv'

# Step 1: Search for the PMID of the articles by title
def search_pmid(article):
    
    title = article['title']
    params = {
        "db": "pubmed",
        "term": f"{title}",
        "retmode": "json",
        "field": "title"
    }

    try:

        # Trying to find the PMID
        response = requests.get(ESEARCH_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if len(data["esearchresult"]["idlist"]) >= 1:
            pmid = data['esearchresult']['idlist'][0]
            print(f"> Found PMID for article: {pmid}")
            return pmid
        
        # Retrying with a shorter title if no results found
        # short_title = ' '.join(title.split()[:8])  # Use first 8
        # print(f"> Retrying with shorter title: {short_title}")
        # params["term"] = f"{short_title}[Title]"

        # response = requests.get(ESEARCH_URL, params=params)
        # response.raise_for_status()
        # data = response.json()

        #if len(data["esearchresult"]["idlist"]) >= 1:
        #    pmid = data['esearchresult']['idlist'][0]
        #    print(f"> Found PMID for article: {pmid}")
        #    return pmid

        print(f"> No PMID found for article.")
        return None

    except requests.exceptions.RequestException as e:
        print(f"> ERROR during request for article: {e}")
        return None
    
ds_pmid = dataset.copy()
for idx, article in ds_pmid.iterrows():
    print(f"[{idx + 1}/{len(ds_pmid)}] Searching PMID for: {article['title']}")
    pmid = search_pmid(article)
    ds_pmid.at[idx, 'pmid'] = pmid

ds_pmid.to_csv(DS_WITH_PMID, index=False)

In [ ]:
print("Number of articles with PMID found:", ds_pmid['pmid'].notnull().sum())

In [ ]:
# Step 2: Fetch article abstract by PMID
def fetch_abstract_by_pmid(pmid):
    params = {
        "db": "pubmed",
        "id": f"{pmid}",
        "retmode": "text",
        "rettype": "abstract",
    }

    try:
        response = requests.get(FETCH_URL, params=params)
        response.raise_for_status()
        print(f"> Fetched abstract!!")
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"> ERROR fetching abstract for PMID '{pmid}': {e}")
        return None
    
ds_pmid_abstract = ds_pmid.copy()
for idx, article in ds_pmid_abstract.iterrows():
    pmid = article['pmid']
    if pd.notnull(pmid):
        print(f"[{idx + 1}/{len(ds_pmid_abstract)}] Fetching abstract for PMID: {pmid}")
        abstract = fetch_abstract_by_pmid(pmid)
        ds_pmid_abstract.at[idx, 'abstract'] = abstract
    else:
        print(f"[{idx + 1}/{len(ds_pmid_abstract)}] No PMID available, skipping abstract fetch.")
        ds_pmid_abstract.at[idx, 'abstract'] = None

In [ ]:
PMID_ABSTRACTS = 'publications_pmid_abstract.csv'

ds_pmid_abstract.to_csv(PMID_ABSTRACTS, index=False)
print("Number of articles with abstract fetched:", ds_pmid_abstract['abstract'].notnull().sum())
print("Number of articles without abstract fetched:", ds_pmid_abstract['abstract'].isnull().sum())

In [2]:
def search_scopus(article_row, api_key):

    try:

        title = article_row.get('title', '').replace('"', '') # Remove quotes for query
        headers = {"Accept": "application/json"}
        params = {
            "query": f"TITLE-ABS-KEY(\"{title}\")",
            "apiKey": api_key
        }
        
        response = requests.get("https://api.elsevier.com/content/search/scopus", 
                                headers=headers, params=params, timeout=10)
        response.raise_for_status() 
        data = response.json()

        if 'search-results' in data and data['search-results']['entry']:
            try:
                return data['search-results']['entry'][0]['prism:url']
            except KeyError:
                return None
        return None

    except requests.exceptions.RequestException as e:
        print(f"> Scopus ERROR: {e}")

In [4]:
ds_elsevier = pd.read_csv("publications_pmid_abstract.csv")

counters = {
    'total_missing': ds_elsevier['abstract'].isnull().sum(),
    'elsevier_found': 0,
    'failed': 0
}

for i, row in ds_elsevier.iterrows():

    if not pd.isnull(row['abstract']):
        continue

    print(f"[{i + 1}/{len(ds_elsevier)}] Searching ELSEVIER for abstract of article: {row['title']}")
    abstract_url = search_scopus(row, "3f5ff36eb8d1d409e3befea2ed2aa2cc")
    
    if abstract_url:
        response = requests.get(abstract_url, 
                                headers={"Accept": "application/json", 
                                         "X-ELS-APIKey": "3f5ff36eb8d1d409e3befea2ed2aa2cc"})
        if response.status_code == 200:
            data = response.json()
            try:
                abstract_text = data['abstracts-retrieval-response']['coredata']['dc:description']
                ds_elsevier.at[i, 'abstract'] = abstract_text
                print("> Found abstract via ELSEVIER!")
                counters['elsevier_found'] += 1
            except KeyError:
                print("> Abstract not found in ELSEVIER response.")
                counters['failed'] += 1
    else:
        print("> Nope :(")
        counters['failed'] += 1

    time.sleep(1) # Polite 1-second delay

ds_elsevier.to_csv('publications_abstract_pubmed_elsevier.csv', index=False)
print("Summary of ELSEVIER abstract search:")
print(f"Total missing abstracts at start: {counters['total_missing']}")
print(f"Abstracts found via ELSEVIER: {counters['elsevier_found']}")
print(f"Failed attempts: {counters['failed']}")

[5/1308] Searching ELSEVIER for abstract of article: Preliminary study of resveratrol content in Aragon red and rose wines
> Found abstract via ELSEVIER!
[7/1308] Searching ELSEVIER for abstract of article: Enhancement of total phenolics and antioxidant properties of some tropical green leafy vegetables by steam cooking
> Found abstract via ELSEVIER!
[11/1308] Searching ELSEVIER for abstract of article: Correlation of tocopherol, tocotrienol, gamma-oryzanol and total polyphenol content in rice bran with different antioxidant capacity assays
> Found abstract via ELSEVIER!
[13/1308] Searching ELSEVIER for abstract of article: High-Performance Liquid-Chromatography of Selected Phenolic-Compounds in Olive Oils
> Found abstract via ELSEVIER!
[14/1308] Searching ELSEVIER for abstract of article: Functional attributes of soybean seeds and products, with reference to isoflavone content and antioxidant activity
> Found abstract via ELSEVIER!
[16/1308] Searching ELSEVIER for abstract of article:

## **Task 2:**

Use the EUtilities tool to search for articles whose content is not relevant to this task. Size of the dataset should be the same of relevant documents.

In [ ]:
def get_articles_pmids_for_title(title, count, api_key=None):
    
    params = {
        "db": "pubmed",
        "term": f"{title}[Title]",
        "retmode": "json",
        "retmax": count,
        "api_key": api_key
    }

    try:
        response = requests.get(ESEARCH_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if 'esearchresult' in data and data['esearchresult']['count'] != '0':
            return data['esearchresult']['idlist']
        else:
            print(f"Found {data['esearchresult']['count']} irrelevant articles.")
            return []

    except requests.exceptions.RequestException as e:
        print(f"Error during request for irrelevant articles: {e}")
        return []


In [ ]:
IRRELEVANT_PUBLICATIONS = 'irrelevant_publications.csv'

irrelevant_pmids_list = get_articles_pmids_for_title("cancer", len(relevant_df), api_key="8e029cc2ba291ed9ee30e494f27c18017408")

irrelevant_abstracts = []
for pmid in irrelevant_pmids_list:

    article_info = {
        'pmid': pmid,
        'abstract': None
    }

    article_info['abstract'] = fetch_abstract_by_pmid(pmid, api_key="8e029cc2ba291ed9ee30e494f27c18017408")
    irrelevant_abstracts.append(article_info)

    # CHANGE ME TO 0.1 IF YOU HAVE AN API KEY
    print("Sleeping for 0.1...")
    time.sleep(0.1)  # Delaying 0.1s to respect NCBI rate limits (3 requests per second)

# Save irrelevant abstracts to a new dataset
irrelevant_df = pd.DataFrame(irrelevant_abstracts)
irrelevant_df.to_csv(IRRELEVANT_PUBLICATIONS, index=False)

In [ ]:
irrelevant_df

## **Task 4:**

Implement the chosen retrieval system using the programming language of their choice. If the information retrieval system is based on machine learning techniques, the student must split the existing datasets (relevant and non-relevant documents) into three distinct groups (training, validation, and testing) to carry out the model training.

**CHOSEN RETRIEVAL SYSTEM:** BioBERT-based Binary Text Classifier

In [ ]:
# Adding target variable 'relevance' 
relevant_df['relevance'] = 1
irrelevant_df['relevance'] = 0

# Combining relevant and irrelevant datasets and maintaining only abstract and relevance columns
features = ['abstract', 'relevance']
combined_df = pd.concat([relevant_df[features], irrelevant_df[features]], ignore_index=True)

# Remove any rows where the abstract is missing (e.g., API fetch failed)
combined_df.dropna(subset=['abstract'], inplace=True)
combined_df.reset_index(drop=True, inplace=True)

# Saving
combined_df.to_csv('combined_publications.csv', index=False)

print("Class distribution:")
print(combined_df['relevance'].value_counts())

combined_df

Following Fine-tuning of BERT for text classification tasks: https://huggingface.co/docs/transformers/en/tasks/sequence_classification

- Train-Test-Validation Split: 80%-10%-10%

In [ ]:
RANDOM_STATE = 42

train_df, test_df = train_test_split(combined_df,
                                     test_size=0.2,
                                     stratify=combined_df["relevance"],
                                     random_state=RANDOM_STATE)

val_df, test_df = train_test_split(test_df,
                                   test_size=0.5,
                                   stratify=test_df["relevance"],
                                   random_state=RANDOM_STATE)

print(f"Training size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

- Convert Pandas DataFrame to HuggingFace Dataset

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

- Tokenization of abstracts using BioBERT tokenizer

In [ ]:
BERT_MODEL_NAME = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

def tokenize(examples):
    return tokenizer(examples["abstract"], 
                     padding="max_length", 
                     truncation=True,
                     max_length=512 # Maximum length for BERT models
                    )

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Renaming the target column to 'labels' as expected by HuggingFace Trainer
train_dataset = train_dataset.rename_column("relevance", "labels")
val_dataset = val_dataset.rename_column("relevance", "labels")
test_dataset = test_dataset.rename_column("relevance", "labels")

- Loading BioBERT model for binary text classification (relevant vs irrelevant)

In [ ]:
id2label = {0: "irrelevant", 1: "relevant"}
label2id = {"irrelevant": 0, "relevant": 1}

model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME, 
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

- Defining evaluation metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, 
                                                               predictions, 
                                                               average="binary",
                                                               zero_division=0)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

- Putting the training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./biobert_pubmed_classifier",

    # Training hyperparameters
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    # Optimiser settings
    weight_decay=0.01,
    
    # Evaluation settings
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,

    # Model selection    
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    # Performance
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,

    seed=RANDOM_STATE,
    push_to_hub=False,
    report_to="none"
)

- Actual training using Trainer API

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

- Evaluating on the test set

In [ ]:
predictions_output = trainer.predict(test_dataset)
predictions = np.argmax(predictions_output.predictions, axis=-1)
true_labels = predictions_output.label_ids

# Calculate all metrics
test_metrics = compute_metrics((predictions_output.predictions, true_labels))

print("\nTest Set Results:")
print(f"Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall:    {test_metrics['recall']:.4f}")
print(f"F1-Score:  {test_metrics['f1']:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(
    true_labels, 
    predictions,
    target_names=['Irrelevant', 'Relevant'],
    digits=4
))

# Confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(true_labels, predictions)
print(cm)
print(f"\nTrue Negatives:  {cm[0][0]} (correctly identified irrelevant)")
print(f"False Positives: {cm[0][1]} (incorrectly marked relevant)")
print(f"False Negatives: {cm[1][0]} (missed relevant papers)")
print(f"True Positives:  {cm[1][1]} (correctly identified relevant)")

- saving the trained model

In [ ]:
model_save_path = './final_biobert_classifier'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)